# KumoRFM Predictive Query Tutorial

## Setup Dataset

## About the Dataset
This tutorial uses Steam gaming data with three tables:
- `users`: Gaming platform users  
- `games`: Video games with pricing/metadata
- `reviews`: User reviews and ratings

We'll predict things like: game discounts, user behavior, and review patterns.

In [ ]:
!pip install kumoai fsspec s3fs

In [ ]:
import os
import kumoai.experimental.rfm as rfm

In [ ]:
if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

In [ ]:
# Initialize a Kumo client with your API key:
KUMO_API_KEY = os.environ.get("KUMO_API_KEY")
rfm.init(api_key=KUMO_API_KEY)

In [ ]:
import pandas as pd

# Take about 20s to download
df_dict = {
    'users': pd.read_csv(f's3://kumo-sdk-public/rfm-datasets/steam_game_sample/users.csv'),
    'games': pd.read_csv(f's3://kumo-sdk-public/rfm-datasets/steam_game_sample/games.csv'),
    'reviews': pd.read_csv(f's3://kumo-sdk-public/rfm-datasets/steam_game_sample/recommendations.csv')
}
df_dict['reviews']['is_recommended'] = df_dict['reviews']['is_recommended'].astype(int)

In [ ]:
# Take a look at the data
df_dict['users'].head(3)

In [ ]:
df_dict['games'].head(3)

In [ ]:
df_dict['reviews'].head(3)

In [ ]:
import kumoai.experimental.rfm as rfm

users_table = rfm.LocalTable(df = df_dict['users'], name = "users").infer_metadata()
games_table = rfm.LocalTable(df = df_dict['games'], name = "games").infer_metadata()
reviews_table = rfm.LocalTable(df = df_dict['reviews'], name = "reviews").infer_metadata()

In [ ]:
games_table.primary_key = "app_id"
games_table['discount'].stype = "numerical"
reviews_table['is_recommended'].stype = "numerical"
reviews_table['helpful'].stype = "numerical"

In [ ]:
from kumoai.experimental.rfm import LocalGraph
graph = LocalGraph(tables=[users_table, games_table, reviews_table])
graph.infer_links()
graph.unlink("users", "reviews", "reviews")
graph.unlink("games", "user_reviews", "reviews")

graph.validate()

In [ ]:
graph.visualize()

In [ ]:
from kumoai.experimental.rfm import KumoRFM

model = KumoRFM(graph)

## Predictive Query Language

**Predictive Query Language (PQL)** is a declarative, SQL-like syntax that lets you frame an entire ML task in a single statement.

---

### Anatomy of a Predictive Query

```
PREDICT <target_expression>
FOR <entity_specification>
[WHERE <filters>]
```

| Component              | Purpose                                                                                                               |
| ---------------------- | --------------------------------------------------------------------------------------------------------------------- |
| **PREDICT**            | Declares the *target*—the value you want the model to output. Can be a column or an aggregation over a future window. |
| **FOR**                | Tells the model *who* (entity) to predict for. In RFM you provide an explicit ID or `IN (...)` list.                  |
| **WHERE** *(optional)* | Filters the *context* used to build features; does **not** limit the entity list you pass in `FOR`.                   |

---

### Writing Queries in Kumo

These are the five steps to prepare writing predictive queries in Kumo.
1. **Choose your entity** - the table and primary-key column you'll predict for.
2. **Define the target** - a raw column *or* an aggregation across a future horizon.
3. **Pin the entity list** - pass a single ID or `IN (...)` list after `FOR`.
4. *(Optional)* **Refine the context** - use `WHERE` with static or temporal filters to restrict feature generation.
5. **Run & fetch** - execute `rfm.query(<your_pql>)` and read the JSON result.

## Use Cases



### Scalar Prediction
Let's start with a simple scalar prediction: predicting the discount amount for a specific game (app_id 113020).

**Important**: The entity column (`games.app_id`) must be the primary key - this tells the model which specific game to predict for.

In [ ]:
query = """PREDICT games.discount

FOR games.app_id = 113020
"""

prediction_result = model.predict(query)
print(prediction_result)

### Boolean & Comparison Operators

Raw predictions are often more useful when converted to actionable decisions. For example, "Will this game have a discount worth my attention (>$5)?"
 We can do so with the binary and comparison operators.

```
=   !=   <   <=   >   >=
AND  OR   NOT
IN   CONTAINS   STARTS WITH   ENDS WITH
LIKE   NOT LIKE
IS NULL   IS NOT NULL
```

Combine them freely inside `WHERE` or as part of a boolean target such as `PREDICT ... = 0`.

The specific binary operator that needs to be used depends on the datatype being compared

---

In [ ]:
query = """PREDICT games.discount > 5

FOR games.app_id = 113020
"""

prediction_result = model.predict(query)
print(prediction_result)

### Time-Based Aggregations
Beyond single values, we often want to predict metrics over time windows: "How many positive reviews will this game get in the next 30 days?

**Aggregation Functions In PQL**

| Function         | Description               | Example                                 |
| ---------------- | ------------------------- | --------------------------------------- |
| `SUM`            | Sum of numeric values     | `SUM(orders.amount, 0, 30)`             |
| `AVG`            | Average of numeric values | `AVG(sensor.temp, 0, 24, hours)`        |
| `COUNT`          | Count of rows             | `COUNT(events.*, 0, 7)`                 |
| `COUNT_DISTINCT` | Count of unique values    | `COUNT_DISTINCT(purchases.item, 0, 90)` |
| `LIST_DISTINCT`  | List of unique values     | `LIST_DISTINCT(articles.tag, 0, 14)`    |
| `MAX` / `MIN`    | Max / Min numeric value   | `MAX(speed.value, 0, 1, hours)`         |

*All aggregations use a sliding window defined by* **`start`** (exclusive) *and* **`end`** (inclusive) *offsets.* Default unit is **days** but you can pass `hours` or `months` as a 4th argument.

Time windows are defined relative to prediction time:
- (0, 30): next 30 days from now
- (-90, 0): past 90 days up to now  
- (-30, 30): 30 days before to 30 days after

---

In [ ]:
query = """PREDICT SUM(reviews.is_recommended, 0, 180, days)

FOR users.user_id = 11227231
"""

prediction_result = model.predict(query)
print(prediction_result)

In [ ]:
query = """PREDICT SUM(reviews.is_recommended, 0, 12, months)

FOR users.user_id = 11227231
"""

prediction_result = model.predict(query)
print(prediction_result)

### Filtering Context with WHERE

The WHERE clause filters which historical data is used to build features - it does not limit which entities you predict for. This improves model quality by focusing on relevant context.
Think of this as providing only certain useful information to the model for better predictions.

For example, we want our model to Only learn from data of games that had recent activity

In [ ]:
q = """PREDICT AVG(reviews.is_recommended, 0, 12, months)
FOR games.app_id = 440
WHERE COUNT(reviews.*, -12, 0, months) > 0"""
print(model.predict(q))

### WHERE inside aggregations
Sometimes, our aggregations shouldn't be over all the values in columns but over selected values, we can then use the ```WHERE``` clause inside an aggregation then.

Suppose we wanted to predict the count of helpful votes of a reviews of a game.

In [ ]:
q = """PREDICT COUNT(reviews.* WHERE reviews.is_recommended=1,0,30, days)
FOR games.app_id=263460"""
print(model.predict(q))

### Link Prediction
So far we've predicted properties of individual entities (like a game's discount or a user's review count). Link prediction goes beyond this to predict relationships between different entities.

For example, we may want to predict the TOP 3 games a user would give reviews for.

In [ ]:
q = """PREDICT LIST_DISTINCT(reviews.app_id, 0, 12, months) RANK TOP 3
FOR users.user_id = 11764552"""
print(model.predict(q, anchor_time=pd.Timestamp('2020-09-20'), num_hops = 6))

### Task Types & Automatic Detection

| Pattern in `PREDICT`                | Task Type                 | Typical Metric  |
| ----------------------------------- | ------------------------- | --------------- |
| Raw numeric column                  | Regression                | MAE, RMSE       |
| Boolean expression (`=`, `>`, etc.) | Binary Classification     | AUC, F1         |
| Categorical column                  | Multiclass / Multilabel   | Accuracy, Top-k |
| `LIST_DISTINCT(...)` + `RANK TOP k` | Link Prediction / Ranking | Recall\@k, MRR  |

The SDK infers the task from your target—no extra parameters required.

4. **Watch for empty windows** - if an entity has *zero* facts in the target window, the training example is dropped.
5. **Start small** - iterate with a few entity IDs before scaling up queries.

### Next Steps

* Explore **evaluation mode**:

  ```python
  metrics = rfm.evaluate("""YOUR PQUERY""")
  ```

Happy querying! 🚀